# Initial Imports and Setup

In [1]:
import time
import os, sys
import numpy as np
from types import SimpleNamespace
from scipy.signal import resample
from scipy.stats import pearsonr, iqr, kurtosis, skew
from sklearn.metrics import r2_score, accuracy_score, f1_score, confusion_matrix
from sklearn.model_selection import GroupKFold

import torch
import torch.nn as nn

import matplotlib.pyplot as plt
import seaborn
%matplotlib inline

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Load Data

In [4]:
def load_data(datapath):
    sub_id = np.genfromtxt(datapath+'demographics.csv', delimiter=',', skip_header=1, usecols=(0,), dtype=str)
    sub_label_str = np.genfromtxt(datapath+'demographics.csv', delimiter=',', skip_header=1, usecols=(2,), dtype=str)
    weight = np.genfromtxt(datapath+'demographics.csv', delimiter=',', skip_header=1, usecols=(7,), dtype=float)
    hy_score = np.genfromtxt(datapath+'demographics.csv', delimiter=',', skip_header=1, usecols=(8,), dtype=float)
    updrs = np.genfromtxt(datapath+'demographics.csv', delimiter=',', skip_header=1, usecols=(9,), dtype=float)
    
    sub_label = [1 if ll == "PD" else 0 for ll in sub_label_str]
    sub_label = np.asarray(sub_label)
    print('# of subjects before excluding', len(sub_id))

    # exclude 2 subjects in PD cuz updrs is not available, and exclude JuC010 cuz sensor data is not available
    exclude_idx = np.logical_or(np.logical_or(np.logical_and(sub_label == 1, np.isnan(updrs)), sub_id == 'Juc010'), np.isnan(weight) )
    sub_id = sub_id[~exclude_idx]
    sub_label_str = sub_label_str[~exclude_idx]
    sub_label = sub_label[~exclude_idx]
    updrs = updrs[~exclude_idx]
    hy_score = hy_score[~exclude_idx]
    weight = weight[~exclude_idx]
    print('# of subjects after excluding', len(sub_id))

    # put 0 to control subjects' updrs score
    updrs[np.isnan(updrs)] = 0.
    hy_score[np.isnan(hy_score)] = 0.
    # updrs_max = np.max(updrs)
    # print(updrs_max)
    # updrs /= updrs_max

    sensor_data = []
    time_stamp = []
    total_data = []
    cnt_n = 0.
    for sub in sub_id:
        # only loading normal walking data
        filename = datapath + sub + '_01.txt'
        time_data = np.genfromtxt(filename, usecols=(0,), dtype=float)
        time_stamp.append(time_data)
        force_data = np.genfromtxt(filename, usecols=(i for i in range(1, 19)), dtype=float)
        sensor_data.append(force_data)
        total_data.extend(force_data)
    sensor_avg = np.mean(total_data, axis=0)
    sensor_std = np.std(total_data, axis=0)

    sensor_data_norm = []
    for i in range(len(sensor_data)):
        # sensor_data_norm.append((dp - sensor_avg) / sensor_std)
        sensor_data_norm.append(sensor_data[i] / weight[i])

    time_stamp = np.asarray(time_stamp)
    sensor_data = np.asarray(sensor_data)
    sensor_data_norm = np.asarray(sensor_data_norm)

    load_data = {'sub_id': sub_id, 'sub_label_str': sub_label_str,
                 'sub_label': sub_label, 'weight': weight,
                 'updrs': updrs, 'hy_score': hy_score,
                 'time_stamp': time_stamp, 'sensor_data': sensor_data}
    return load_data

In [5]:
datapath = '/content/drive/My Drive/Colab Notebooks/data/'
fs = 100

data = load_data(datapath)

# save data
output = {'data': data}
np.savez('/content/drive/My Drive/Colab Notebooks/data/data.npz', **output)

# of subjects before excluding 166
# of subjects after excluding 160
[ 64.6632845   58.64447628  51.48497729  62.98713781  33.33331342
  66.91266928  75.9107824   37.0625552   63.6954244   57.8639058
  53.57801153  65.0148925   32.72398001  65.44760867  76.60633762
  37.86128391 450.99919618 452.79144445] [103.65497811  88.55219404  80.39144361  81.51970075  52.96860482
  83.10623679  97.70354357  54.3708706  102.86710686  86.54377873
  83.31593468  82.12089313  50.15001627  82.41026856 100.83862514
  50.92223436 402.95257745 401.11341393]


# Movement Element Decomposition

In [6]:
def find_conseq_subarr(arr):
    '''
    Find subarrays in which elements are consecutive
    
    :param arr: array of elements
    :return conseq_subarr: list that contain consecutive elements
    :return conseq_indices
    find_conseq_subarr([1,2,3,4,6,7,8,10])
    return [[1, 2, 3, 4], [6, 7, 8], [10]]
    '''
    conseq_subarr, conseq_indices = [], [0]
    currentIdx = 0
    for i in range(1, len(arr)):
        if arr[i] - arr[currentIdx] == i - currentIdx:
            continue
        else:
            conseq_subarr.append(arr[currentIdx:i])
            currentIdx = i
    conseq_subarr.append(arr[currentIdx:])
    for subarr in conseq_subarr:
        conseq_indices.append(conseq_indices[-1] + len(subarr))
    return conseq_subarr, conseq_indices

def decompose(data):
    non_zero_indices = np.where(data!=0)[0]
    conseq_subarr, conseq_indices = find_conseq_subarr(non_zero_indices)
    return conseq_subarr

def nan_padded_matrix(arrs):
    height = len(arrs)
    max_width = max([a.shape[0] for a in arrs])
    result = np.full((height, max_width), np.nan, dtype='float64')
    for i in range(height):
        width = arrs[i].shape[0]
        result[i, 0:width] = arrs[i]
    return result

In [7]:
sub_id, sub_label = [], []
sub_updrs = []
duration = []
sensor_data = []
sensor_data_norm = []
for i in range(len(data['sub_id'])):
    sId = data['sub_id'][i]
    sLabel = data['sub_label'][i]
    sData = data['sensor_data'][i]
    time_stamp = data['time_stamp'][i]
    print('Subject ID :%s' % sId)
    for j in range(sData.shape[1]):
        indices = decompose(sData[:, j])
        for idx in indices:
            sub_id.append(sId)
            sub_label.append(sLabel)
            sub_updrs.append(data['updrs'])
            duration.append(time_stamp[idx[-1]] - time_stamp[idx[0]])
            sensor_data.append(sData[idx, j])
            sensor_data_norm.append(resample(sData[idx, j] / np.mean(sData[idx, j]), 50))

sub_id = np.array(sub_id)
sub_label = np.array(sub_label)
sub_updrs = np.array(sub_updrs)
duration = np.array(duration)
sensor_data = nan_padded_matrix(sensor_data)
sensor_data_norm = np.array(sensor_data_norm)

Subject ID :GaPt03
Subject ID :GaPt05
Subject ID :GaPt06
Subject ID :GaPt07
Subject ID :GaPt08
Subject ID :GaPt09
Subject ID :GaPt12
Subject ID :GaPt13
Subject ID :GaPt14
Subject ID :GaPt15
Subject ID :GaPt16
Subject ID :GaPt17
Subject ID :GaPt18
Subject ID :GaPt19
Subject ID :GaPt20
Subject ID :GaPt21
Subject ID :GaPt22
Subject ID :GaPt24
Subject ID :GaPt25
Subject ID :GaPt26
Subject ID :GaPt27
Subject ID :GaPt28
Subject ID :GaPt29
Subject ID :GaPt30
Subject ID :GaPt31
Subject ID :GaPt32
Subject ID :GaPt33
Subject ID :JuPt01
Subject ID :JuPt02
Subject ID :JuPt03
Subject ID :JuPt04
Subject ID :JuPt05
Subject ID :JuPt06
Subject ID :JuPt07
Subject ID :JuPt08
Subject ID :JuPt09
Subject ID :JuPt10
Subject ID :JuPt11
Subject ID :JuPt12
Subject ID :JuPt13
Subject ID :JuPt14
Subject ID :JuPt15
Subject ID :JuPt16
Subject ID :JuPt17
Subject ID :JuPt18
Subject ID :JuPt19
Subject ID :JuPt20
Subject ID :JuPt21
Subject ID :JuPt22
Subject ID :JuPt23
Subject ID :JuPt24
Subject ID :JuPt25
Subject ID :

In [11]:
output = {'sub_id': sub_id,
          'sub_label': sub_label,
          'updrs': sub_updrs,
          'duration': duration,
          'sensor_data': sensor_data,
          'sensor_data_norm': sensor_data_norm}
np.savez('/content/drive/My Drive/Colab Notebooks/mvmt_elem.npz', **output)

# Feature Extraction

In [12]:
def rms(data, axis, keepdims=False):
    return np.sqrt(np.mean(np.square(data), axis=axis, keepdims=keepdims))

def featureExtraction(mvmt_elem):
    '''
    Extract features on movement element level
    Parameters:
        - mvmt_elem: movement element, NUM_SAMPLE x SAMPLE_RATE
        - vel: original vel, contains nan
        - acc: original acceleration, contains nan
        - dist: traveled distance of mvmt_elem, NUM_SAMPLE x 1
    Returns:
        - features: every row is a sample
                    every column is a feature
        - feature_names: feature name
    '''
    NUM_SAMPLE, SMAPLE_RATE = mvmt_elem.shape
    features = np.zeros((NUM_SAMPLE, 1))
    feature_names = []
    
    ###############################################################
    # Statistic Features of Movement Element
    ###############################################################
    # median
    features[:, 0] = np.median(mvmt_elem, axis=1)
    feature_names.append("median_mvmt elem")
    
    # std. dev
    features = np.append(features, np.std(mvmt_elem, axis=1, keepdims=True), axis=1)
    feature_names.append("std. dev_mvmt elem")
    
    # rms
    features = np.append(features, rms(mvmt_elem, axis=1, keepdims=True), axis=1)
    feature_names.append("rms_mvmt elem")
    
    # iqr
    features = np.append(features, iqr(mvmt_elem, axis=1, keepdims=True), axis=1)
    feature_names.append("iqr_mvmt elem")
    
    # maximum value
    features = np.append(features, np.max(mvmt_elem, axis=1, keepdims=True), axis=1)
    # features = np.append(features, np.max(np.absolute(mvmt_elem), axis=1, keepdims=True), axis=1)
    feature_names.append("max_mvmt elem")
    
    # kurtosis
    features = np.append(features, kurtosis(mvmt_elem, axis=1)[:, np.newaxis], axis=1)
    feature_names.append("kurtosis_mvmt elem")
    
    # skewness
    features = np.append(features, skew(mvmt_elem, axis=1)[:, np.newaxis], axis=1)
    feature_names.append("skew_mvmt elem")
    
    assert features.shape[1] == len(feature_names)
    return features, feature_names

In [13]:
def featureAggregation(subjects, features, feature_names):
    agMethod = ["mean", "std", "iqr", "10per", "median", "90per"]
    
    k = sorted(set(subjects))
    num_feature = features.shape[1]
    agFeatures = np.zeros((len(k), num_feature*len(agMethod)))

    for i, j in enumerate(k):
        # i: order, j: (subject, trial)
        agFeatures[i,0*num_feature:1*num_feature] = np.mean(features[subjects==j, :], axis=0)
        agFeatures[i,1*num_feature:2*num_feature] = np.std(features[subjects==j, :], axis=0)
        agFeatures[i,2*num_feature:3*num_feature] = iqr(features[subjects==j, :], axis=0)
        agFeatures[i,3*num_feature:4*num_feature] = np.percentile(features[subjects==j, :], 10, axis=0)
        agFeatures[i,4*num_feature:5*num_feature] = np.median(features[subjects==j, :], axis=0)
        agFeatures[i,5*num_feature:6*num_feature] = np.percentile(features[subjects==j, :], 90, axis=0)
        
    agNames = []
    for method in agMethod:
        for name in feature_names:
            agNames.append("%s of %s" % (method, name))

    assert agFeatures.shape[1] == len(agNames)
    return k, agFeatures, agNames

In [14]:
# extract feature
features = []
labels = []
updrs = []
for sId in np.unique(sub_id):
    indices = sub_id == sId
    mvmt_elem_feature, feature_names = featureExtraction(sensor_data_norm[indices])
    features.extend(mvmt_elem_feature)
    labels.append(sub_label[indices][0])
    updrs.append(sub_updrs[indices][0])
features = np.array(features)
labels = np.array(labels)
updrs = np.array(updrs)
subjID, features, feature_names = featureAggregation(sub_id, features, feature_names)
print('# of sameple: %d' % features.shape[0])
print('# of features: %d' % features.shape[1])

# of sameple: 160
# of features: 42


In [15]:
output = {'subjID': subjID,
          'features': features,
          'labels': labels,
          'updrs': data['updrs'],
          'feature_names': feature_names}
np.savez('/content/drive/My Drive/Colab Notebooks/features.npz', **output)